In [44]:
import pandas as pd
import os 
import fastdup
import shutil


def search_parameter(df: pd.DataFrame = None, parameter: dict = None) -> pd.DataFrame:
    
    if not len(parameter):
        return df
    
    df_columns = df.columns
    
    # validamos que el parametro este en el df, en caso de que no lo quitamos
    for key in parameter.copy():
        if key not in df_columns:
            
            print(f"{key} no se encuentra en el df.")
            del parameter[key]
    
    if not len(parameter):
        print("Ninguno de los parámetros proporcionados se encuentran en el df por lo tanto lo retornamos como estaba.")
        return df
    
    # validamos los valores de busqueda
    for key,value in parameter.copy().items():
        
        if isinstance(value, (str, int, float, bool)):
            
            search = df[ df[key] == value]
            if not len(search):
                print(f"No se encontro en valor '{value}' en la columna '{key}' por lo tanto no se va a buscar.")
                del parameter[key]
        
        elif isinstance(value, list):
            
            search = df[df[key].isin(value)]
            if not len(search):
                print(f"No se encontro los valores '{value}' en la columna '{key}' por lo tanto no se va a buscar.")
                del parameter[key]
        
        elif isinstance(value, dict):
            print(f"No se aceptan parametros de tipo {type(value).__name__}")
            del parameter[key]
    
    if not len(parameter):
        print("Ninguno de los parámetros proporcionados se encuentran en el df por lo tanto lo retornamos como estaba.")
        return df
    
    # si todo sale bien empezamos a realizar las busquedas en el df
    df_copy = df
    
    for key, value in parameter.items():
        
        if isinstance(value, (str, int, float, bool)):
            
            df_copy = df_copy[ df_copy[key] == value]
        
        elif isinstance(value, list):
            
            df_copy = df_copy[df_copy[key].isin(value)]
    
    # retornamos el df filtrado
    return df_copy


# df = pd.read_json("../data/json/Myntra__Marianfer_Cruz_20231219_195028.success.json")
# print(len(df))
# parameter = {
#     "color": ["White", "Brown"],
#     "brand": "BUCIK",
# }

# df = search_parameter(df, parameter)
# print(len(df))

In [ ]:
path_origin: str = "../data/json/New_collector_20231214_154733.success.json"
path_alternative: str = "../data/json/Myntra__Marianfer_Cruz_20231219_195028.success.json"

origin_df = pd.read_json(path_origin)
alternative_df = pd.read_json(path_alternative)

In [ ]:
path_img_origin = [os.path.join("../trash/fastdup/tmp/ajio-myntra/origin/20231214", path) for path in search_parameter(origin_df, {"sku":464087281009})["product_images"].to_list()[0]]
path_img_alternative = [os.path.join("../trash/fastdup/tmp/ajio-myntra/alternative/20231219", path) for path in search_parameter(alternative_df, {"sku": 22050204})["product_images"].to_list()[0]]

images = path_img_origin + path_img_alternative

fd = fastdup.create("trash/test")
fd.run(images, threshold= 0.1, overwrite= True)
similarity = fd.similarity()

In [46]:
def copiar_archivos(lista_archivos, directorio_destino):
    for archivo in lista_archivos:
        shutil.copy(archivo, directorio_destino)

copiar_archivos(path_img_origin, "../trash/test/origin")
copiar_archivos(path_img_alternative, "../trash/test/alternative")